In [1]:
import gzip
import csv
import pytz
import pandas as pd
import numpy as np
import io
import csv
import requests
import urllib.request
import boto3
import datetime
import s3fs
import os
import time
import googlemaps
from googlemaps import Client
import logging
from botocore.vendored import requests
from math import sin, cos, sqrt, atan2, radians

In [2]:
OBJ_S3 = boto3.resource('s3')
S_BUCKET = 'tdp-opendata-demo'  
LOGGER = logging.getLogger()
LOGGER.setLevel(logging.INFO)

In [3]:
def read_od_from_s3(df):
    '''
    read od from s3
    '''
    client = boto3.client('s3')
    obj_op = client.get_object(Bucket='tdp-opendata-demo', Key='TDP_opendata/{}.csv'.format(df))
    df =pd.read_csv(obj_op['Body'])
    return df

In [19]:
def read_all_from_s3(diff,now,diff_key,now_key):
    '''
    read all data under folder from s3
    '''
    name=[]
    time=[] ##get s3 file name & create time
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('kardi-data-raw-temp-01')
    for obj in bucket.objects.filter(Prefix=diff_key):
        name.append(obj.key)
        time.append(obj.last_modified)
        raw_data_diff= pd.DataFrame(
        {'name': name,
         'time': time,})##clear the data name must remove file path,only keep file name
    for obj in bucket.objects.filter(Prefix=now_key):
        name.append(obj.key)
        time.append(obj.last_modified)
        raw_data_now= pd.DataFrame(
        {'name': name,
         'time': time,})                    
        #time must be covert to GTM time zone
        raw_data=pd.concat([raw_data_diff,raw_data_now])
        #raw_data['name2']=raw_data['name'].str[28:]
        raw_data['time2']=raw_data.time.dt.tz_convert('Etc/GMT-8')
        raw_data_filter=raw_data[(raw_data['time2']>diff.strftime("%Y-%m-%d %H:%M")) &
                         (raw_data['time2']<=now.strftime("%Y-%m-%d %H:%M"))]
        raw_data_filter=raw_data_filter.drop_duplicates(keep='first',inplace=False)
        fs = s3fs.S3FileSystem() #get object by raw_data_filter's file name 
        obj_list=[]
    for obj in raw_data_filter['name']:
        
        obj_list.append(fs.open('s3://tdp-opendata-demo/{}'.format(obj)))
        df_list=[]  #open object and combine then 
    
    for file in obj_list:
        df = pd.read_csv(file)
        df_list.append(df)
        final_df = pd.concat(df_list).drop_duplicates().reset_index(drop=True) #remove duplicate     
    return final_df

In [58]:
    name=[]
    time=[] ##get s3 file name & create time
    s3 = boto3.resource('s3')
    bucket = s3.Bucket('kardi-data-raw-temp-01')
    for obj in bucket.objects.filter(Prefix='2018/11/05/07'):
        name.append(obj.key)
        time.append(obj.last_modified)
        raw_data= pd.DataFrame(
        {'name': name,
         'time': time,})##clear the data name must remove file path,only keep file name


In [59]:
        raw_data['time2']=raw_data.time.dt.tz_convert('Etc/GMT-8')
        raw_data_filter=raw_data[(raw_data['time2']>diff.strftime("%Y-%m-%d %H:%M")) &
                         (raw_data['time2']<=now.strftime("%Y-%m-%d %H:%M"))]
        raw_data_filter=raw_data_filter.drop_duplicates(keep='first',inplace=False)
        fs = s3fs.S3FileSystem() #get object by raw_data_filter's file name 

In [61]:
    obj_list=[]
    for obj in raw_data_filter['name']:  
        obj_list.append(fs.open('s3://kardi-data-raw-temp-01/{}'.format(obj)))
        
    df_list=[]  #open object and combine then 
    for file in obj_list:
        df = pd.read_csv(file, compression='gzip',sep='|')
        df_list.append(df)
        final_df = pd.concat(df_list)       

In [62]:
final_df[final_df['vehicle_id']=='000CBF149141']

,serial_no,event_type,vehicle_id,event_time,total_mileage,event_lon,event_lat,coolant,fuel_pressure,im_pressure,...,move_time,idling_time,ng_walk_start,s1_driving_safety_score,s3_energy_saving_driving_score,stable_speed_time,raw_total_fuel_analysis,avg_fuel_store_analysis,fuel_total_analysis,peak
0,20181105070024000CBF149141_em7d6bfi,11,000CBF149141,2018-11-05 07:00:24.0,0.000000,121.590866,25.063059,89.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,100.00,0.0,5.401967,0.360131,0.000000,1
0,20181105070054000CBF149141_9h41i5l7,11,000CBF149141,2018-11-05 07:00:54.0,0.000000,121.590860,25.063095,89.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,100.00,0.0,5.401967,0.360131,0.000000,1
0,20181105070124000CBF149141_6he5l751,11,000CBF149141,2018-11-05 07:01:24.0,106.111115,121.590770,25.065142,85.0,0.0,0.0,...,0.0,0.0,0.0,66.666664,76.25,3.0,15.273315,1.018221,0.104212,1
0,20181105070154000CBF149141_h92lm1fk,11,000CBF149141,2018-11-05 07:01:54.0,98.888885,121.590700,25.067053,85.0,0.0,0.0,...,0.0,0.0,2.0,100.000000,106.25,7.0,5.569374,0.371292,0.266338,1
0,20181105070224000CBF149141_k57fk91f,11,000CBF149141,2018-11-05 07:02:24.0,22.777775,121.590770,25.067566,87.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,106.25,4.0,6.025051,0.401670,0.056708,1
0,20181105070254000CBF149141_fi0nkf30,11,000CBF149141,2018-11-05 07:02:54.0,95.555550,121.590805,25.069365,85.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,106.25,14.0,7.635433,0.509029,0.187721,1
0,20181105070324000CBF149141_62365hfb,11,000CBF149141,2018-11-05 07:03:24.0,109.444435,121.589920,25.071365,86.0,0.0,0.0,...,0.0,0.0,1.0,100.000000,100.00,7.0,12.907555,0.860504,0.127186,1
0,20181105070454000CBF149141_3bf2j84h,11,000CBF149141,2018-11-05 07:04:54.0,0.000000,121.589910,25.071384,88.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,100.00,0.0,5.401967,0.360131,0.000000,1
0,20181105070524000CBF149141_4b055561,11,000CBF149141,2018-11-05 07:05:24.0,95.555560,121.589935,25.073025,85.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,106.25,6.0,13.777260,0.918484,0.104036,1
0,20181105070554000CBF149141_6ij36g1m,11,000CBF149141,2018-11-05 07:05:54.0,185.555570,121.590310,25.076590,85.0,0.0,0.0,...,0.0,0.0,0.0,100.000000,106.25,13.0,6.826974,0.455132,0.407697,1
